# Diet Optimization with cuOpt Python API

This notebook demonstrates how to solve the classic diet optimization problem using the cuOpt Python API. The problem involves selecting foods to meet nutritional requirements while minimizing cost.

## Problem Description

We need to select quantities of different foods to:
- Meet minimum and maximum nutritional requirements
- Minimize total cost
- Satisfy additional constraints (like limiting dairy servings)

The nutrition guidelines are based on USDA Dietary Guidelines for Americans, 2005.


## Environment Setup

First, let's check if we have a GPU available and install necessary dependencies.


In [ ]:
# Check for GPU availability
!nvidia-smi


In [ ]:
# Install cuOpt if not already installed
# Uncomment the following line if running in Google Colab or similar environment
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu12 # For cuda 12
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu13 # For cuda 13


## Import Required Libraries


In [ ]:
import numpy as np
import pandas as pd
from cuopt.linear_programming.problem import Problem, VType, sense, LinearExpression
from cuopt.linear_programming.solver_settings import SolverSettings
import time


## Problem Data Setup

Define the nutrition guidelines, food costs, and nutritional values for each food item.


In [ ]:
# Nutrition guidelines based on USDA Dietary Guidelines for Americans, 2005
# http://www.health.gov/DietaryGuidelines/dga2005/

# minimum and maximum values for each category
categories = {
    "calories": {
        "min": 1800,
        "max": 2200
    },
    "protein": {
        "min": 91,
        "max": float('inf')
    },
    "fat": {
        "min": 0,
        "max": 65
    },
    "sodium": {
        "min": 0,
        "max": 1779
    }
}



In [ ]:
# Food costs per serving
food_costs = {
    "hamburger": 2.49,
    "chicken": 2.89,
    "hot dog": 1.50,
    "fries": 1.89,
    "macaroni": 2.09,
    "pizza": 1.99,
    "salad": 2.49,
    "milk": 0.89,
    "ice cream": 1.59
}

# Nutrition values for each food (per serving)
nutrition_data = {
    "hamburger": [410, 24, 26, 730],
    "chicken": [420, 32, 10, 1190],
    "hot dog": [560, 20, 32, 1800],
    "fries": [380, 4, 19, 270],
    "macaroni": [320, 12, 10, 930],
    "pizza": [320, 15, 12, 820],
    "salad": [320, 31, 12, 1230],
    "milk": [100, 8, 2.5, 125],
    "ice cream": [330, 8, 10, 180]
}


In [ ]:
# Create a DataFrame for better visualization
nutrition_df = pd.DataFrame(nutrition_data, index=categories.keys()).T
nutrition_df.columns = [f"{cat} (per serving)" for cat in categories.keys()]
print("Nutritional Values per Serving:")
print(nutrition_df)

## Problem Formulation

Now we'll create the optimization problem using the cuOpt Python API as MILP. The problem has:
- **Variables**: Amount of each food to buy (continuous, non-negative)
- **Objective**: Minimize total cost
- **Constraints**: Meet nutritional requirements (minimum and maximum bounds)

Since these are price per serving, you need to have a whole number for number of product that will be used.


In [ ]:
# Create the optimization problem
problem = Problem("diet_optimization")

# Add decision variables for each food (amount to buy)
buy_vars = {}
for food_name in food_costs:
    # Using integer type for amount of food to buy since serving needs to be whole number
    # And this converts the problem to MILP
    var = problem.addVariable(name=f"{food_name}", vtype=VType.INTEGER, lb=0.0, ub=float('inf'))
    buy_vars[food_name] = var

print(f"Created {len(buy_vars)} decision variables for foods")
print(f"Variables: {[var.getVariableName() for var in buy_vars.values()]}")


In [ ]:
objective_expr = LinearExpression([], [], 0.0)

for var in buy_vars.values():
    if food_costs[var.getVariableName()] != 0:  # Only include non-zero coefficients
        objective_expr += var * food_costs[var.getVariableName()]

# Set objective function: minimize total cost
problem.setObjective(objective_expr, sense.MINIMIZE)


In [ ]:
# Add nutrition constraints
constraint_names = []

for i, category in enumerate(categories):
    # Calculate total nutrition from all foods for this category
    nutrition_expr = LinearExpression([], [], 0.0)
    
    for food_name in food_costs: 
        nutrition_value = nutrition_data[food_name][i]
        if nutrition_value != 0:  # Only include non-zero coefficients
            nutrition_expr += buy_vars[food_name] * nutrition_value
    
    # Add constraint: min_nutrition[i] <= nutrition_expr <= max_nutrition[i]
    min_val = categories[category]["min"]
    max_val = categories[category]["max"]
    
    if max_val == float('inf'):
        # Only lower bound constraint
        constraint = problem.addConstraint(nutrition_expr >= min_val, name=f"min_{category}")
        constraint_names.append(f"min_{category}")
    else:
        # Range constraint (both lower and upper bounds)
        constraint = problem.addConstraint(nutrition_expr >= min_val, name=f"min_{category}")
        constraint_names.append(f"min_{category}")
        constraint = problem.addConstraint(nutrition_expr <= max_val, name=f"max_{category}")
        constraint_names.append(f"max_{category}")

print(f"Added {len(constraint_names)} nutrition constraints")
print(f"Constraints: {constraint_names}")


## Solver Configuration and Solution

Configure the solver settings and solve the optimization problem.


In [ ]:
# Configure solver settings
settings = SolverSettings()
settings.set_parameter("time_limit", 60.0)  # 60 second time limit
settings.set_parameter("log_to_console", True)  # Enable solver logging
settings.set_parameter("method", 0)  # Use default method

print("Solver configured with 60-second time limit")


In [ ]:
# Solve the problem
print("Solving diet optimization problem...")
print(f"Problem type: {'MIP' if problem.IsMIP else 'LP'}")

start_time = time.time()
problem.solve(settings)
solve_time = time.time() - start_time

print(f"\nSolve completed in {solve_time:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")


In [ ]:
def print_solution():
    """Print the optimal solution in a readable format"""
    if problem.Status.name == "Optimal" or problem.Status.name == "FeasibleFound":
        print(f"\nOptimal Solution Found!")
        print(f"Total Cost: ${problem.ObjValue:.2f}")
        print("\nFood Purchases:")
        
        total_cost = 0
        for var in buy_vars.values():
            amount = var.getValue()
            if amount > 0.0001:  # Only show foods with significant amounts
                food_cost = amount * food_costs[var.getVariableName()]
                total_cost += food_cost
                print(f"  {var.getVariableName()}: {amount:.3f} servings (${food_cost:.2f})")
        
        print(f"\nTotal Cost: ${total_cost:.2f}")
        
        # Check nutritional intake
        print("\nNutritional Intake:")
        for i, category in enumerate(categories):
            total_nutrition = 0
            for var in buy_vars.values():
                amount = var.getValue()
                nutrition_value = nutrition_data[var.getVariableName()][i]
                total_nutrition += amount * nutrition_value
            
            min_req = categories[category]["min"]
            max_req = categories[category]["max"]
            
            # Check constraints with tolerance for floating point precision
            tolerance = 1e-6
            min_satisfied = total_nutrition >= (min_req - tolerance)
            max_satisfied = (max_req == float('inf')) or (total_nutrition <= (max_req + tolerance))
            status = "✓" if (min_satisfied and max_satisfied) else "✗"
            
            if max_req == float('inf'):
                print(f"  {category}: {total_nutrition:.1f} (min: {min_req}) {status}")
            else:
                print(f"  {category}: {total_nutrition:.1f} (min: {min_req}, max: {max_req}) {status}")
    else:
        print(f"No optimal solution found. Status: {problem.Status.name}")

print_solution()


## Adding Additional Constraints

Now let's demonstrate how to add additional constraints to the existing model. We'll add a constraint to limit dairy servings to at most 6.


In [ ]:
# Create LinearExpression for dairy constraint
dairy_expr = buy_vars["milk"] + buy_vars["ice cream"]

dairy_constraint = problem.addConstraint(dairy_expr <= 6, name="limit_dairy")

In [ ]:
# Solve the problem again with the new constraint
print("\nSolving with dairy constraint...")
print(f"Problem now has {problem.NumVariables} variables and {problem.NumConstraints} constraints")

start_time = time.time()
problem.solve(settings)
solve_time = time.time() - start_time

print(f"\nSolve completed in {solve_time:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")


## Solution Comparison

Let's compare the solutions before and after adding the dairy constraint to see the impact.


In [ ]:
# Display the new solution
print_solution()

## Conclusion

This notebook demonstrated how to:

1. **Formulate a diet optimization problem** using the cuOpt Python API
2. **Set up decision variables** for food quantities
3. **Define an objective function** to minimize total cost
4. **Add nutritional constraints** with both lower and upper bounds
5. **Solve the optimization problem** using cuOpt's high-performance solver
6. **Add additional constraints** to the existing model
7. **Analyze and compare solutions** before and after constraint modifications

The cuOpt Python API provides a clean, intuitive interface for building and solving optimization problems, making it easy to model complex real-world scenarios like diet optimization.

### Key Benefits of cuOpt:
- **High Performance**: GPU-accelerated solving for large-scale problems
- **Easy to Use**: Intuitive Python API similar to other optimization libraries
- **Flexible**: Support for both LP and MIP problems
- **Scalable**: Handles problems with thousands of variables and constraints efficiently



SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.